# Zadanie domowe - łączenie danych!

Wykonujesz zlecenia dla firmy, która chce żebyśmy połączyli dane z trzech różnych źródeł. 

1. Pierwsze źródło danych to dane o klientach. Na szczęście zespół, który zajmuje się klientami, wie co robi i wszystkie dane dostępne są w bazie danych (`zad_domowe__clients.db` w tabeli `clients`).
2. Drugie źródło danych to dane o produktach. W tym przypadku dane dostępne są w pliku CSV (`zad_domowe__products.csv`).
3. Ostatnie źródło danych to dane o zamówieniach / fakturach. Tutaj niestety mamy problem. Okazuje się, że klient wszystkie faktury ma w postaci zrzutów z ekranu (pliki `zad_domowe__invoice_1.png`, `zad_domowe__invoice_2.png` etc.). 

Twoim zadaniem jest połączenie tych danych w jeden plik CSV, który będzie zawierał informacje o kliencie, produkcie i fakturze. Przygotuj plik CSV, który zawiera tylko niezbędne kolumny, nie ma żadnych kolumn powtórzonych. 

Jak rezultat wyciągania danych z faktur chcemy mieć w każdym wierszu informacje o zakupionym produkcie, ilości, kliencie, cenie jednostkowej.

Początek kodu znajdziesz w tym notebooku.

In [46]:
# RESTARTUJ KERNEL PO URUCHOMIENIU
!pip install --quiet instructor pydantic openai

In [47]:
from pathlib import Path
import pandas as pd
import sqlite3
import base64
from getpass import getpass
from datetime import date
import instructor
from pydantic import BaseModel
from openai import OpenAI

In [73]:
# będziemy potrzebować klucza API do OpenAI żeby odczytać dane z faktur
openai_key = getpass("OpenAI key: ")

OpenAI key:  ········


## Czytamy dane klientów

In [74]:
with sqlite3.connect('data_zd/raw_zd/zad_domowe__clients.db') as conn:
    clients_df = pd.read_sql_query('SELECT * FROM clients', conn)

clients_df

,id,name,email,phone
0,1,Carla Ramsey,frypeter@example.net,(508)998-5166x12326
1,2,Howard Mcgee,jacobpatterson@example.org,001-669-353-6118x978
2,3,Eric Moore,lejacob@example.org,923-681-4646
3,4,Laura Beard,westdominique@example.org,(291)558-0966
4,5,Joshua Santiago,sheilamedina@example.com,+1-929-417-0571x74051
...,...,...,...,...
95,96,Heather Ortega,xmartin@example.org,+1-355-627-6736x91833
96,97,Olivia Barnes,dpacheco@example.net,9317151015
97,98,Tara Cabrera,geraldjames@example.net,+1-984-996-5056
98,99,Alexandra Parker,katherineharrison@example.com,001-823-577-8901


In [75]:
clients_df.sample(10)

,id,name,email,phone
5,6,John Parsons,mhouse@example.com,+1-775-381-9658x664
40,41,Ryan Humphrey,derek13@example.net,(931)240-8149
8,9,Alexander Smith,debra94@example.org,919.678.4577
21,22,Gregory Perry,michellejones@example.com,995-610-5186x033
9,10,Antonio Jones,ryangoodman@example.org,846-767-9939x22814
23,24,Elizabeth Callahan,derrick78@example.org,214.778.8425x1222
84,85,Joshua Wilson,cgonzalez@example.org,643-519-7048x80995
70,71,Denise Mahoney,smithamanda@example.org,915-553-8585x373
92,93,Jessica Wheeler,suttonnancy@example.org,816.477.6232x6407
28,29,Victoria Baker,amy77@example.com,+1-202-589-9429x541


In [76]:
clients_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   id      100 non-null    int64 
 1   name    100 non-null    object
 2   email   100 non-null    object
 3   phone   100 non-null    object
dtypes: int64(1), object(3)
memory usage: 3.3+ KB


In [77]:
clients_df.nunique()

id       100
name     100
email    100
phone    100
dtype: int64

In [78]:
clients_df.isnull().sum()

id       0
name     0
email    0
phone    0
dtype: int64

In [79]:
clients_df = clients_df.rename(columns={'id': 'customer_id', 'name': 'customer_name'})
clients_df

,customer_id,customer_name,email,phone
0,1,Carla Ramsey,frypeter@example.net,(508)998-5166x12326
1,2,Howard Mcgee,jacobpatterson@example.org,001-669-353-6118x978
2,3,Eric Moore,lejacob@example.org,923-681-4646
3,4,Laura Beard,westdominique@example.org,(291)558-0966
4,5,Joshua Santiago,sheilamedina@example.com,+1-929-417-0571x74051
...,...,...,...,...
95,96,Heather Ortega,xmartin@example.org,+1-355-627-6736x91833
96,97,Olivia Barnes,dpacheco@example.net,9317151015
97,98,Tara Cabrera,geraldjames@example.net,+1-984-996-5056
98,99,Alexandra Parker,katherineharrison@example.com,001-823-577-8901


## Czytamy dane produktów

In [80]:
products_df = pd.read_csv('data_zd/raw_zd/zad_domowe__products.csv', sep=';')
products_df

,id,name,price
0,1,telewizor,85
1,2,laptop,47
2,3,smartfon,60
3,4,tablet,40
4,5,słuchawki,40
5,6,kamera,92
6,7,router,48
7,8,myszka,30
8,9,klawiatura,65
9,10,pendrive,47


In [81]:
products_df.sample(10)

,id,name,price
6,7,router,48
17,18,snowboard,85
15,16,ochraniacze,46
2,3,smartfon,60
10,11,rower,69
16,17,narty,55
11,12,hulajnoga,44
0,1,telewizor,85
18,19,kurtka,12
7,8,myszka,30


In [82]:
products_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20 entries, 0 to 19
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   id      20 non-null     int64 
 1   name    20 non-null     object
 2   price   20 non-null     int64 
dtypes: int64(2), object(1)
memory usage: 612.0+ bytes


In [83]:
products_df.nunique()

id       20
name     20
price    16
dtype: int64

In [84]:
products_df.isnull().sum()

id       0
name     0
price    0
dtype: int64

In [85]:
products_df = products_df.rename(columns={'id': 'product_id', 'name': 'product_name'})
products_df

,product_id,product_name,price
0,1,telewizor,85
1,2,laptop,47
2,3,smartfon,60
3,4,tablet,40
4,5,słuchawki,40
5,6,kamera,92
6,7,router,48
7,8,myszka,30
8,9,klawiatura,65
9,10,pendrive,47


## Czytamy dane faktur

In [86]:
DATA_PATH = Path('data_zd')
RAW_DATA_PATH = Path('data_zd') / 'raw_zd'
PROCESSED_DATA_PATH = Path('data_zd') / 'processed_zd'

In [93]:
# na szczęście mamy już strukturę którą chcemy pobrać z faktur
class InvoiceInfoItem(BaseModel):
    product_name: str
    product_id: int
    quantity: int
    price: float


class InvoiceInfo(BaseModel):
    company_name: str
    customer_id: int
    customer_name: str
    invoice_number: int
    date: date
    # mamy tutaj do czynienia z listą elementów, więc danymi zagnieżdżonymi!
    items: list[InvoiceInfoItem]


instructor_openai_client = instructor.from_openai(OpenAI(api_key=openai_key))
orders = []

In [95]:
def prepare_image_for_open_ai(image_path):
    with open(image_path, "rb") as f:
        image_data = base64.b64encode(f.read()).decode('utf-8')
        return f"data:image/png;base64,{image_data}"


In [96]:
for image_path in sorted(RAW_DATA_PATH.glob("zad_domowe__invoice_*.png")):
    print(f"Processing {image_path}")
    with open(image_path, "rb") as f:
        image_data = base64.b64encode(f.read()).decode('utf-8')

    invoice_info = instructor_openai_client.chat.completions.create(
        model="gpt-4o",
        response_model=InvoiceInfo,
        messages=[
            {
                "role": "user",
                "content": [
                    {
                        "type": "text",
                        "text": "Pobierz szczegóły faktur",
                    },
                    {
                        "type": "image_url",
                        "image_url": {
                            "url": prepare_image_for_open_ai(image_path),
                            "detail": "high"
                        },
                    },
                ],
            },
        ],
    )
    with open(PROCESSED_DATA_PATH / f"{image_path.stem}.json", "w") as f:
        f.write(invoice_info.model_dump_json())

    invoice_data = invoice_info.model_dump()
    for item in invoice_data["items"]:
        order = {
            "company_name": invoice_data["company_name"],
            "customer_id": invoice_data["customer_id"],
            "customer_name": invoice_data["customer_name"],
            "invoice_number": invoice_data["invoice_number"],
            "date": invoice_data["date"],
            "product_name": item["product_name"],
            "product_id": item["product_id"],
            "quantity": item["quantity"],
            "price": item["price"],
        }
        orders.append(order)

orders_df = pd.DataFrame(orders)
        

Processing data_zd/raw_zd/zad_domowe__invoice_1.png
Processing data_zd/raw_zd/zad_domowe__invoice_10.png
Processing data_zd/raw_zd/zad_domowe__invoice_2.png
Processing data_zd/raw_zd/zad_domowe__invoice_3.png
Processing data_zd/raw_zd/zad_domowe__invoice_4.png
Processing data_zd/raw_zd/zad_domowe__invoice_5.png
Processing data_zd/raw_zd/zad_domowe__invoice_6.png
Processing data_zd/raw_zd/zad_domowe__invoice_7.png
Processing data_zd/raw_zd/zad_domowe__invoice_8.png
Processing data_zd/raw_zd/zad_domowe__invoice_9.png


In [147]:
orders_df

,company_name,customer_id,customer_name,invoice_number,date,product_name,product_id,quantity,price
0,Firma XYZ,3,Eric Moore,2,2024-05-25,router,7,3,48.0
1,Firma XYZ,3,Eric Moore,2,2024-05-25,narty,17,5,55.0
2,Firma XYZ,92,Valerie Stevens,11,2024-05-02,słuchawki,5,1,40.0
3,Firma XYZ,60,Troy Alexander,3,2024-05-19,ochraniacze,16,4,46.0
4,Firma XYZ,78,William Velazquez,4,2024-03-26,rolki,14,3,11.0
5,Firma XYZ,33,Steven Jackson,5,2024-07-23,pendrive,10,4,47.0
6,Firma XYZ,33,Steven Jackson,5,2024-07-23,laptop,2,2,47.0
7,Firma XYZ,8,Ronald Callahan,6,2024-03-14,słuchawki,5,4,40.0
8,Firma XYZ,8,Ronald Callahan,6,2024-03-14,ochraniacze,16,2,46.0
9,Firma XYZ,4,Laura Beard,7,2024-06-24,telewizor,1,2,85.0


In [98]:
orders_df.sample(10)

,company_name,customer_id,customer_name,invoice_number,date,product_name,product_id,quantity,price
5,Firma XYZ,33,Steven Jackson,5,2024-07-23,pendrive,10,4,47.0
12,Firma XYZ,63,Alexis Johnson,8,2024-02-21,spodnie,20,1,14.0
2,Firma XYZ,92,Valerie Stevens,11,2024-05-02,słuchawki,5,1,40.0
8,Firma XYZ,8,Ronald Callahan,6,2024-03-14,ochraniacze,16,2,46.0
11,Firma XYZ,63,Alexis Johnson,8,2024-02-21,router,7,1,48.0
7,Firma XYZ,8,Ronald Callahan,6,2024-03-14,słuchawki,5,4,40.0
15,Firma XYZ,44,Kimberly Marquez,10,2024-03-14,klawiatura,9,2,65.0
9,Firma XYZ,4,Laura Beard,7,2024-06-24,telewizor,1,2,85.0
13,Firma XYZ,100,Johnny Franklin,9,2024-01-26,spodnie,20,2,14.0
1,Firma XYZ,3,Eric Moore,2,2024-05-25,narty,17,5,55.0


In [102]:
orders_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16 entries, 0 to 15
Data columns (total 9 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   company_name    16 non-null     object 
 1   customer_id     16 non-null     int64  
 2   customer_name   16 non-null     object 
 3   invoice_number  16 non-null     int64  
 4   date            16 non-null     object 
 5   product_name    16 non-null     object 
 6   product_id      16 non-null     int64  
 7   quantity        16 non-null     int64  
 8   price           16 non-null     float64
dtypes: float64(1), int64(4), object(4)
memory usage: 1.3+ KB


In [100]:
orders_df.nunique()

company_name       1
customer_id       10
customer_name     10
invoice_number    10
date               9
product_name      10
product_id        10
quantity           5
price              9
dtype: int64

In [105]:
orders_df['date'] = pd.to_datetime(orders_df['date'])
orders_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16 entries, 0 to 15
Data columns (total 9 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   company_name    16 non-null     object        
 1   customer_id     16 non-null     int64         
 2   customer_name   16 non-null     object        
 3   invoice_number  16 non-null     int64         
 4   date            16 non-null     datetime64[ns]
 5   product_name    16 non-null     object        
 6   product_id      16 non-null     int64         
 7   quantity        16 non-null     int64         
 8   price           16 non-null     float64       
dtypes: datetime64[ns](1), float64(1), int64(4), object(3)
memory usage: 1.3+ KB


In [106]:
orders_df.isnull().sum()

company_name      0
customer_id       0
customer_name     0
invoice_number    0
date              0
product_name      0
product_id        0
quantity          0
price             0
dtype: int64

## Łączenie danych

In [128]:
orders_products_df = orders_df.merge(products_df, on='product_id', how='outer')
orders_products_df

,company_name,customer_id,customer_name,invoice_number,date,product_name_x,product_id,quantity,price_x,product_name_y,price_y
0,Firma XYZ,3.0,Eric Moore,2.0,2024-05-25,router,7,3.0,48.0,router,48
1,Firma XYZ,63.0,Alexis Johnson,8.0,2024-02-21,router,7,1.0,48.0,router,48
2,Firma XYZ,3.0,Eric Moore,2.0,2024-05-25,narty,17,5.0,55.0,narty,55
3,Firma XYZ,92.0,Valerie Stevens,11.0,2024-05-02,słuchawki,5,1.0,40.0,słuchawki,40
4,Firma XYZ,8.0,Ronald Callahan,6.0,2024-03-14,słuchawki,5,4.0,40.0,słuchawki,40
5,Firma XYZ,60.0,Troy Alexander,3.0,2024-05-19,ochraniacze,16,4.0,46.0,ochraniacze,46
6,Firma XYZ,8.0,Ronald Callahan,6.0,2024-03-14,ochraniacze,16,2.0,46.0,ochraniacze,46
7,Firma XYZ,78.0,William Velazquez,4.0,2024-03-26,rolki,14,3.0,11.0,rolki,11
8,Firma XYZ,33.0,Steven Jackson,5.0,2024-07-23,pendrive,10,4.0,47.0,pendrive,47
9,Firma XYZ,33.0,Steven Jackson,5.0,2024-07-23,laptop,2,2.0,47.0,laptop,47


In [129]:
orders_products_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 26 entries, 0 to 25
Data columns (total 11 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   company_name    16 non-null     object        
 1   customer_id     16 non-null     float64       
 2   customer_name   16 non-null     object        
 3   invoice_number  16 non-null     float64       
 4   date            16 non-null     datetime64[ns]
 5   product_name_x  16 non-null     object        
 6   product_id      26 non-null     int64         
 7   quantity        16 non-null     float64       
 8   price_x         16 non-null     float64       
 9   product_name_y  26 non-null     object        
 10  price_y         26 non-null     int64         
dtypes: datetime64[ns](1), float64(4), int64(2), object(4)
memory usage: 2.4+ KB


In [130]:
orders_products_df = orders_products_df.drop('product_name_x', axis=1)

In [131]:
orders_products_df = orders_products_df.drop('price_x', axis=1)

In [132]:
orders_products_df = orders_products_df.rename(columns={'product_name_y': 'product_name', 'price_y': 'price'})

In [133]:
orders_products_df.nunique()

company_name       1
customer_id       10
customer_name     10
invoice_number    10
date               9
product_id        20
quantity           5
product_name      20
price             16
dtype: int64

In [136]:
orders_products_df.isnull().sum()

company_name      10
customer_id       10
customer_name     10
invoice_number    10
date              10
product_id         0
quantity          10
product_name       0
price              0
dtype: int64

In [139]:
orders_products_df = orders_products_df.fillna({
    'company_name': 'NO SALES',
    'customer_id': 0,
    'customer_name': 'NO SALES',
    'invoice_number': 0,
    'quantity': 0})
orders_products_df
    
    

,company_name,customer_id,customer_name,invoice_number,date,product_id,quantity,product_name,price
0,Firma XYZ,3.0,Eric Moore,2.0,2024-05-25,7,3.0,router,48
1,Firma XYZ,63.0,Alexis Johnson,8.0,2024-02-21,7,1.0,router,48
2,Firma XYZ,3.0,Eric Moore,2.0,2024-05-25,17,5.0,narty,55
3,Firma XYZ,92.0,Valerie Stevens,11.0,2024-05-02,5,1.0,słuchawki,40
4,Firma XYZ,8.0,Ronald Callahan,6.0,2024-03-14,5,4.0,słuchawki,40
5,Firma XYZ,60.0,Troy Alexander,3.0,2024-05-19,16,4.0,ochraniacze,46
6,Firma XYZ,8.0,Ronald Callahan,6.0,2024-03-14,16,2.0,ochraniacze,46
7,Firma XYZ,78.0,William Velazquez,4.0,2024-03-26,14,3.0,rolki,11
8,Firma XYZ,33.0,Steven Jackson,5.0,2024-07-23,10,4.0,pendrive,47
9,Firma XYZ,33.0,Steven Jackson,5.0,2024-07-23,2,2.0,laptop,47


In [169]:
company_df = pd.merge(orders_products_df, clients_df, on='customer_id', how='left')
company_df

,company_name,customer_id,customer_name_x,invoice_number,date,product_id,quantity,product_name,price,customer_name_y,email,phone
0,Firma XYZ,3.0,Eric Moore,2.0,2024-05-25,7,3.0,router,48,Eric Moore,lejacob@example.org,923-681-4646
1,Firma XYZ,63.0,Alexis Johnson,8.0,2024-02-21,7,1.0,router,48,Alexis Johnson,campbellkatrina@example.org,(311)679-3346
2,Firma XYZ,3.0,Eric Moore,2.0,2024-05-25,17,5.0,narty,55,Eric Moore,lejacob@example.org,923-681-4646
3,Firma XYZ,92.0,Valerie Stevens,11.0,2024-05-02,5,1.0,słuchawki,40,Valerie Stevens,xlowe@example.com,757-943-1579x98346
4,Firma XYZ,8.0,Ronald Callahan,6.0,2024-03-14,5,4.0,słuchawki,40,Ronald Callahan,audrey56@example.com,+1-727-668-2720x085
5,Firma XYZ,60.0,Troy Alexander,3.0,2024-05-19,16,4.0,ochraniacze,46,Troy Alexander,nturner@example.net,+1-212-349-3992x430
6,Firma XYZ,8.0,Ronald Callahan,6.0,2024-03-14,16,2.0,ochraniacze,46,Ronald Callahan,audrey56@example.com,+1-727-668-2720x085
7,Firma XYZ,78.0,William Velazquez,4.0,2024-03-26,14,3.0,rolki,11,William Velazquez,lisa25@example.net,(765)602-1572x949
8,Firma XYZ,33.0,Steven Jackson,5.0,2024-07-23,10,4.0,pendrive,47,Steven Jackson,jeremy07@example.com,481.258.2434
9,Firma XYZ,33.0,Steven Jackson,5.0,2024-07-23,2,2.0,laptop,47,Steven Jackson,jeremy07@example.com,481.258.2434


In [170]:
company_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 26 entries, 0 to 25
Data columns (total 12 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   company_name     26 non-null     object        
 1   customer_id      26 non-null     float64       
 2   customer_name_x  26 non-null     object        
 3   invoice_number   26 non-null     float64       
 4   date             16 non-null     datetime64[ns]
 5   product_id       26 non-null     int64         
 6   quantity         26 non-null     float64       
 7   product_name     26 non-null     object        
 8   price            26 non-null     int64         
 9   customer_name_y  16 non-null     object        
 10  email            16 non-null     object        
 11  phone            16 non-null     object        
dtypes: datetime64[ns](1), float64(3), int64(2), object(6)
memory usage: 2.6+ KB


In [171]:
company_df = company_df.drop('customer_name_y', axis=1)
company_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 26 entries, 0 to 25
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   company_name     26 non-null     object        
 1   customer_id      26 non-null     float64       
 2   customer_name_x  26 non-null     object        
 3   invoice_number   26 non-null     float64       
 4   date             16 non-null     datetime64[ns]
 5   product_id       26 non-null     int64         
 6   quantity         26 non-null     float64       
 7   product_name     26 non-null     object        
 8   price            26 non-null     int64         
 9   email            16 non-null     object        
 10  phone            16 non-null     object        
dtypes: datetime64[ns](1), float64(3), int64(2), object(5)
memory usage: 2.4+ KB


In [172]:
company_df = company_df.rename(columns={'customer_name_x': 'customer_name'})
company_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 26 entries, 0 to 25
Data columns (total 11 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   company_name    26 non-null     object        
 1   customer_id     26 non-null     float64       
 2   customer_name   26 non-null     object        
 3   invoice_number  26 non-null     float64       
 4   date            16 non-null     datetime64[ns]
 5   product_id      26 non-null     int64         
 6   quantity        26 non-null     float64       
 7   product_name    26 non-null     object        
 8   price           26 non-null     int64         
 9   email           16 non-null     object        
 10  phone           16 non-null     object        
dtypes: datetime64[ns](1), float64(3), int64(2), object(5)
memory usage: 2.4+ KB


In [173]:
company_df.isnull().sum()

company_name       0
customer_id        0
customer_name      0
invoice_number     0
date              10
product_id         0
quantity           0
product_name       0
price              0
email             10
phone             10
dtype: int64

In [174]:
company_df = company_df.fillna({
    'email': 'NO SALES',
    'phone': 'NO SALES' })
company_df

,company_name,customer_id,customer_name,invoice_number,date,product_id,quantity,product_name,price,email,phone
0,Firma XYZ,3.0,Eric Moore,2.0,2024-05-25,7,3.0,router,48,lejacob@example.org,923-681-4646
1,Firma XYZ,63.0,Alexis Johnson,8.0,2024-02-21,7,1.0,router,48,campbellkatrina@example.org,(311)679-3346
2,Firma XYZ,3.0,Eric Moore,2.0,2024-05-25,17,5.0,narty,55,lejacob@example.org,923-681-4646
3,Firma XYZ,92.0,Valerie Stevens,11.0,2024-05-02,5,1.0,słuchawki,40,xlowe@example.com,757-943-1579x98346
4,Firma XYZ,8.0,Ronald Callahan,6.0,2024-03-14,5,4.0,słuchawki,40,audrey56@example.com,+1-727-668-2720x085
5,Firma XYZ,60.0,Troy Alexander,3.0,2024-05-19,16,4.0,ochraniacze,46,nturner@example.net,+1-212-349-3992x430
6,Firma XYZ,8.0,Ronald Callahan,6.0,2024-03-14,16,2.0,ochraniacze,46,audrey56@example.com,+1-727-668-2720x085
7,Firma XYZ,78.0,William Velazquez,4.0,2024-03-26,14,3.0,rolki,11,lisa25@example.net,(765)602-1572x949
8,Firma XYZ,33.0,Steven Jackson,5.0,2024-07-23,10,4.0,pendrive,47,jeremy07@example.com,481.258.2434
9,Firma XYZ,33.0,Steven Jackson,5.0,2024-07-23,2,2.0,laptop,47,jeremy07@example.com,481.258.2434


In [175]:
company_df = company_df[['date', 'product_id', 'product_name', 'quantity', 'price', 'company_name', 'customer_id', 'customer_name', 'invoice_number', 'email', 'phone']]      

In [177]:
company_df

,date,product_id,product_name,quantity,price,company_name,customer_id,customer_name,invoice_number,email,phone
0,2024-05-25,7,router,3.0,48,Firma XYZ,3.0,Eric Moore,2.0,lejacob@example.org,923-681-4646
1,2024-02-21,7,router,1.0,48,Firma XYZ,63.0,Alexis Johnson,8.0,campbellkatrina@example.org,(311)679-3346
2,2024-05-25,17,narty,5.0,55,Firma XYZ,3.0,Eric Moore,2.0,lejacob@example.org,923-681-4646
3,2024-05-02,5,słuchawki,1.0,40,Firma XYZ,92.0,Valerie Stevens,11.0,xlowe@example.com,757-943-1579x98346
4,2024-03-14,5,słuchawki,4.0,40,Firma XYZ,8.0,Ronald Callahan,6.0,audrey56@example.com,+1-727-668-2720x085
5,2024-05-19,16,ochraniacze,4.0,46,Firma XYZ,60.0,Troy Alexander,3.0,nturner@example.net,+1-212-349-3992x430
6,2024-03-14,16,ochraniacze,2.0,46,Firma XYZ,8.0,Ronald Callahan,6.0,audrey56@example.com,+1-727-668-2720x085
7,2024-03-26,14,rolki,3.0,11,Firma XYZ,78.0,William Velazquez,4.0,lisa25@example.net,(765)602-1572x949
8,2024-07-23,10,pendrive,4.0,47,Firma XYZ,33.0,Steven Jackson,5.0,jeremy07@example.com,481.258.2434
9,2024-07-23,2,laptop,2.0,47,Firma XYZ,33.0,Steven Jackson,5.0,jeremy07@example.com,481.258.2434


company_df = company_df.sort_values(by='date')
company_df

## Transakcje są posortowane wg dat sprzedaży. Tabela obejmuje wszystkie produkty, również te niesprzedane.

In [181]:
company_df.to_csv('AK_zadanie_domowe_modul_5.csv', index=False)